In [81]:
import numpy as np
N, T, dX, dU,dV = 10, 100,26, 7, 7

l = np.zeros(T)
lu = np.zeros((T, dU))
lv = np.zeros((T, dV))
lx = np.zeros((T, dX))
luu = np.zeros((T, dU, dU))
lvv = np.zeros((T, dU, dV))
lxx = np.zeros((T, dX, dX))
lux = np.zeros((T, dU, dX))
lvx = np.zeros((T, dV, dX))

# Compute cost.
cs = np.zeros((N, T))  # see algorithm_utils.py
cc = np.zeros((N, T))   # Cost estimate constant term.
cv = np.zeros((N, T, dX+dU+dV))    # Cost estimate vector term.
Cm = np.zeros((N, T, dX+dU+dV, dX+dU+dV)) # Cost estimate matrix term.

term1 = np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])]
term2 = np.c_[lux, luu, lvv]
cv_temp = np.c_[lx, lu,  lv]
term3 = np.c_[lvx, lvv]
Cm_ord = np.concatenate(
                (np.c_[lxx, np.transpose(lux, [0, 2, 1])], np.c_[lux, luu]),
                axis=1
            )
Cm_temp = np.concatenate(
            (np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])], np.c_[lux, luu, lvv]),
            axis=1
        )   
Cm2 = np.concatenate((Cm_temp, np.c_[lvx, lvv, luu],), axis=1)
# # print('term1: {}, Cm: {}, term2: {}, term3: {}'.format(term1.shape, Cm.shape, term2.shape, term3.shape))
print('Cm : {} temp'.format(Cm_temp.shape), np.c_[lxx, np.transpose(lux, [0, 2, 1])].shape)
print(np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])].shape)
print(np.c_[lux, luu, lvv].shape)
print(np.c_[lvx, lvv, luu].shape)
print('Cm temp: ', Cm_temp.shape)
print('Cm2 shape', Cm2.shape)
for n in range(N):
    cc[n, :] = l
    cs[n, :] = l
    cv[n,:,:] = np.c_[lx, lu,  lv]
    Cm_temp = np.concatenate(
            (np.c_[lxx, np.transpose(lux, [0, 2, 1]), np.transpose(lvx, [0, 2, 1])], np.c_[lux, luu, lvv]),
            axis=1
        )     
    Cm[n, :, :, :] = np.concatenate((Cm_temp, np.c_[lvx, lvv, luu],), axis=1)

print('cs: {}, cv: {}, Cm: {}'.format(cs.shape, cv.shape, Cm.shape))

('Cm : (100, 33, 40) temp', (100, 26, 33))
(100, 26, 40)
(100, 7, 40)
(100, 7, 40)
('Cm temp: ', (100, 33, 40))
('Cm2 shape', (100, 40, 40))
cs: (10, 100), cv: (10, 100, 40), Cm: (10, 100, 40, 40)


In [18]:

26+7+7


40

In [26]:
gamma, T = 1e-2, 100
b = np.diag(2 * gamma * np.ones(7))
b = np.tile(b, [T, 1, 1])
b.shape

(100, 7, 7)

In [29]:
c = np.empty_like(b)
c = [1, 2, 3,4]
if isinstance(c, list):
    print(isinstance(c, list))
else:
    print('not a list')

True
